In [1]:
from utils import uniprotRetrieve
import pandas as pd
import subprocess
from tqdm import tqdm
import tensorflow as tf #Make sure version 1.3.0 is installed
import numpy as np
import os

def cdhit(inputFastaFile, identity=0.9, outputDir=""):
    if outputDir and not os.path.exists(outputDir):
        os.makedirs(outputDir)
    outputFile=inputFastaFile.split("/")[-1].replace(".fasta","_cdhit.fasta")
    outputFilePath="{}{}".format(outputDir,outputFile)
    command = "cd-hit -i {} -o {} -c {}  -n 3 -M 1500".format(inputFastaFile,outputFilePath,identity)
    cmd = subprocess.Popen(command,
                           shell=True,
                           stdin=subprocess.PIPE,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT)
    cmd.communicate()
    return outputFilePath

""" Read in model to generate Unirep vectors"""
# Sync relevant weight files
!aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
# Import the mLSTM babbler model
from unirep import babbler64 as babbler

# Where model weights are stored.
MODEL_WEIGHT_PATH = "./64_weights"

batch_size = 12
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:461: FutureWarnin

# 1. Download data

### 1.1.1. Cytoplasmic proteins

In [ ]:
"""Download sequences and identifiers"""
QUERY="taxonomy:Gammaproteobacteria (locations:(location:cytoplasm) OR locations:(location:cytosol)) NOT annotation:(type:signal)"
FORMAT="tab"
FILENAME="cytoplasm.tab"
COLUMNS="id,sequence"
uniprotRetrieve(FILENAME, format=FORMAT, query=QUERY, columns=COLUMNS)

CYTOPLASM = pd.read_csv(FILENAME,sep="\t")
# Write fasta file 
fastaFile = open("cytoplasm.fasta","w")
for i,row in tqdm(CYTOPLASM.iterrows()):
    entry = row["Entry"]
    sequence = row["Sequence"]
    fastaFile.write(">{}\n{}\n".format(entry,sequence))
fastaFile.close()

In [ ]:
""" Use cdhit to obtain dataset of no more than 50 percent sequences identit """
CYTOPLASM_CDHIT = cdhit("cytoplasm.fasta",identity=0.5)

### 1.1.2 Periplasmic proteins

In [ ]:
QUERY="taxonomy:Gammaproteobacteria locations:(location:periplasm) annotation:(type:signal)"
FORMAT="tab"
FILENAME="periplasm.tab"
COLUMNS="id,sequence,feature(SIGNAL)"
uniprotRetrieve(FILENAME, format=FORMAT, query=QUERY, columns=COLUMNS)

PERIPLASM = pd.read_csv(FILENAME,sep="\t")
PERIPLASM["newStart"]=PERIPLASM["Signal peptide"].apply(lambda x : int(x.split(";")[0].split("..")[-1]))
# Write fasta file with and without fasta 
withSP = open("periplasm_with_sp.fasta","w")
withoutSP = open("periplasm_without_sp.fasta","w")
for i,row in tqdm(PERIPLASM.iterrows()):
    entry = row["Entry"]
    sequence = row["Sequence"]
    newStart = row["newStart"]
    withSP.write(">{}\n{}\n".format(entry,sequence))
    withoutSP.write(">{}\n{}\n".format(entry,sequence[newStart:]))
withSP.close()
withoutSP.close()

In [ ]:
""" Use cdhit to obtain dataset of no more than 50 percent sequences identity (without SP) """
PERIPLASM_WITHOUT_SP_CDHIT = cdhit("periplasm_without_sp.fasta",identity=0.5)

In [ ]:
""" Use cdhit to obtain dataset of no more than 50 percent sequences identity (with SP) """
PERIPLASM_WITH_SP_CDHIT = cdhit("periplasm_with_sp.fasta",identity=0.5)

### 1.1.3 Generate unirep vector

In [ ]:
""" Generate vectors for periplamic proteins without SP """

with open("periplasm_without_sp_cdhit.fasta") as f:
    lines = [line.strip() for line in f.readlines()]

periplasm=dict()
for line in lines:
    if line.startswith(">"):
        ID = line.replace(">","")
        periplasm[ID]=dict()
        periplasm[ID]["seq"]=""
    else:
        periplasm[ID]["seq"]+=line

FILE="periplasm_without_sp.unirep"
if not os.path.exists(FILE):
    with open(FILE, 'w'): pass      

with open(FILE) as f:
    IDS_DONE = [LINE.strip().replace(">","") for LINE in f.readlines() if LINE.startswith(">")]
IDS_TODO = [ID for ID in periplasm.keys() if ID not in IDS_DONE]

total = len(IDS_TODO)
i=1
for ID in IDS_TODO:
    seq = periplasm[ID]["seq"]
    repres = b.get_rep(seq)
    #periplasm[ID]["unirep"]= repres
    print(i,"/",total)
    i+=1
    with open(FILE,"ab") as f:
        f.write(bytes(">"+ID+"\n", 'utf-8'))
        np.savetxt(f,repres)

1 / 3457
2 / 3457
3 / 3457
4 / 3457
5 / 3457
6 / 3457
7 / 3457
8 / 3457
9 / 3457
10 / 3457
11 / 3457
12 / 3457
13 / 3457
14 / 3457
15 / 3457
16 / 3457
17 / 3457
18 / 3457
19 / 3457
20 / 3457
21 / 3457
22 / 3457
23 / 3457
